# Checking EEG code

Some reference code (to be checked):

- https://github.com/AliAmini93/ADHDeepNet/tree/main

**ADHD data**: https://ieee-dataport.org/open-access/eeg-data-adhd-control-children

## Data loading

Dataframe `adhd_df` will contain EEG data for all patients diagnosed with ADHD (for the 19 electrodes), plus one column with the ID of the patient. 

Dataframe `control_df` will be the equivalent for the control patients. 

In [1]:
# Data loading
from pathlib import Path
import scipy.io as sio

import numpy as np
import pandas as pd

data_dir = Path.home() / Path("MyData/EEG_ADHD")

In [2]:
# Channel labels and electrode positions
channel_labels = {
    0: "Fp1",
    1: "Fp2",
    2: "F3",
    3: "F4",
    4: "C3",
    5: "C4",
    6: "P3",
    7: "P4",
    8: "O1",
    9: "O2",
    10: "F7",
    11: "F8",
    12: "T7",
    13: "T8",
    14: "P7",
    15: "P8",
    16: "Fz",
    17: "Cz",
    18: "Pz",
}

electrode_positions = {
    "Fp1": (-18, 0.511, 0.95, 0.309, -0.0349, 18, -2, 1),
    "Fp2": (18, 0.511, 0.95, -0.309, -0.0349, -18, -2, 1),
    "F7": (-54, 0.511, 0.587, 0.809, -0.0349, 54, -2, 1),
    "F3": (-39, 0.333, 0.673, 0.545, 0.5, 39, 30, 1),
    "Fz": (0, 0.256, 0.719, 0, 0.695, 0, 44, 1),
    "F4": (39, 0.333, 0.673, -0.545, 0.5, -39, 30, 1),
    "F8": (54, 0.511, 0.587, -0.809, -0.0349, -54, -2, 1),
    "T7": (-90, 0.511, 6.12e-17, 0.999, -0.0349, 90, -2, 1),
    "C3": (-90, 0.256, 4.4e-17, 0.719, 0.695, 90, 44, 1),
    "Cz": (90, 0, 3.75e-33, -6.12e-17, 1, -90, 90, 1),
    "C4": (90, 0.256, 4.4e-17, -0.719, 0.695, -90, 44, 1),
    "T8": (90, 0.511, 6.12e-17, -0.999, -0.0349, -90, -2, 1),
    "P7": (-126, 0.511, -0.587, 0.809, -0.0349, 126, -2, 1),
    "P3": (-141, 0.333, -0.673, 0.545, 0.5, 141, 30, 1),
    "Pz": (180, 0.256, -0.719, -8.81e-17, 0.695, -180, 44, 1),
    "P4": (141, 0.333, -0.673, -0.545, 0.5, -141, 30, 1),
    "P8": (126, 0.511, -0.587, -0.809, -0.0349, -126, -2, 1),
    "O1": (-162, 0.511, -0.95, 0.309, -0.0349, 162, -2, 1),
    "O2": (162, 0.511, -0.95, -0.309, -0.0349, -162, -2, 1),
}

# Sampling Frequency Hz
Sampling_Frequency = 128

# Set the chunk size
chunk_size = 512

In [3]:
def split_into_chunks(df, chunk_size, initial_chunk_number=0):
    # Calculate the number of full chunks
    n_chunks = len(df) // chunk_size
    chunks = []

    # Split into chunks and keep track of the chunk number
    for i in range(n_chunks):
        chunk = df.iloc[i * chunk_size : (i + 1) * chunk_size].copy()  # Get the chunk
        chunk["chunk_number"] = (
            initial_chunk_number + i
        )  # Add the chunk number as a new column
        chunks.append(chunk)

    # Concatenate the chunks back together
    return pd.concat(chunks, ignore_index=True), n_chunks

In [4]:
def load_data(data_dirs):
    data_list = []
    chunked_data_list = []

    chunk_index = 0
    for directory in data_dirs:
        # print(f"Loading data from {directory}")

        for filepath in directory.glob("*.mat"):
            mat = sio.loadmat(filepath)
            key = list(mat.keys())[-1]  # Get the last key (the id of the patient)
            eeg_data = mat[key]

            # Convert the EEG data to a DataFrame
            # Assuming the EEG data is a 2D array (time x channels)
            df = pd.DataFrame(eeg_data)
            df = df.rename(columns=channel_labels)
            # Add a column to identify the source
            df["subject_id"] = key

            # print(f"Loaded data for patient {key}; chunks start at {chunk_index}")
            chucked_df, chunks = split_into_chunks(df, chunk_size, chunk_index)
            chunk_index += chunks

            # Append the DataFrame to the list
            data_list.append(df)
            chunked_data_list.append(chucked_df)

        # Concatenate all DataFrames in the list into a single DataFrame
        full_eeg_df = pd.concat(data_list, ignore_index=True)
        chunked_eeg_df = pd.concat(chunked_data_list, ignore_index=True)

    return full_eeg_df, chunked_eeg_df

In [5]:
adhd_dir1 = data_dir / Path("ADHD_part1")
adhd_dir2 = data_dir / Path("ADHD_part2")
adhd_df, adhd_chunks_df = load_data([adhd_dir1, adhd_dir2])

control_dir1 = data_dir / Path("Control_part1")
control_dir2 = data_dir / Path("Control_part2")
control_df, control_chunks_df = load_data([control_dir1, control_dir2])

### Some sanity checks on the data

In [6]:
adhd_subjects = adhd_df["subject_id"].unique()
control_subjects = control_df["subject_id"].unique()
intersection = [item for item in adhd_subjects if item in control_subjects]


num_patients = len(adhd_subjects) + len(control_subjects)
num_data_points = adhd_df.shape[0] + control_df.shape[0]
num_chunks = 0
num_chunks2 = 0

print(f"Number of ADHD subjects: {len(adhd_subjects)}")
print(f"Number of Control subjects: {len(control_subjects)}")
print(f"Number of subjects in both groups: {len(intersection)}")

# print(adhd_df.info())
# print(control_df.info())

# print(adhd_df.describe())
# print(control_df.describe())

chunck_size = 512

num_chunks = 0
for patient in adhd_subjects:
    data_points = adhd_df[adhd_df["subject_id"] == patient].shape[0]
    num_chunks += data_points // chunck_size

print(f"ADHD - total number of data points: {adhd_df.shape[0]:,}")
print(f"ADHD - total number of chunks of size {chunck_size}: {num_chunks}")

num_chunks = 0
for patient in control_subjects:
    data_points = control_df[control_df["subject_id"] == patient].shape[0]
    num_chunks += data_points // chunck_size

print(f"Control - total number of data points: {control_df.shape[0]:,}")
print(f"Control - total number of chunks of size {chunck_size}: {num_chunks}")

# adhd_df[adhd_df['subject_id']==adhd_subjects[0]].drop(columns=['subject_id']).plot(subplots=True, figsize=(10, 10), title='ADHD for subject ' + adhd_subjects[0])

Number of ADHD subjects: 61
Number of Control subjects: 60
Number of subjects in both groups: 0
ADHD - total number of data points: 1,207,069
ADHD - total number of chunks of size 512: 2330
Control - total number of data points: 959,314
Control - total number of chunks of size 512: 1843


In [8]:
print("*****\nADHD subjects\n*****")
for subject in adhd_subjects:
    print(
        f"Subject {subject} has {adhd_df[adhd_df['subject_id']==subject].shape[0]} data points"
    )

print("*****\nControl subjects\n*****")
for subject in control_subjects:
    print(
        f"Subject {subject} has {control_df[control_df['subject_id']==subject].shape[0]} data points"
    )

*****
ADHD subjects
*****
Subject v36p has 17401 data points
Subject v20p has 35328 data points
Subject v40p has 20097 data points
Subject v21p has 16574 data points
Subject v6p has 17561 data points
Subject v173 has 24241 data points
Subject v37p has 9286 data points
Subject v10p has 14304 data points
Subject v30p has 21663 data points
Subject v1p has 12258 data points
Subject v27p has 28880 data points
Subject v31p has 11679 data points
Subject v12p has 17604 data points
Subject v28p has 27612 data points
Subject v24p has 16385 data points
Subject v3p has 33570 data points
Subject v32p has 18049 data points
Subject v33p has 29217 data points
Subject v25p has 9894 data points
Subject v29p has 24193 data points
Subject v18p has 25003 data points
Subject v22p has 12100 data points
Subject v34p has 19555 data points
Subject v14p has 17562 data points
Subject v38p has 24695 data points
Subject v39p has 18177 data points
Subject v8p has 15776 data points
Subject v15p has 43252 data points


In [9]:
# subject = adhd_subjects[0]
# df = adhd_df[adhd_df['subject_id']==subject].drop(columns=['subject_id'])
# df.plot(subplots=True, figsize=(10, 10), title='ADHD for subject ' + subject)

### Organise data into chunks

In [9]:
# Let's put all the data together (we will use the "chunked" data)
adhd_chunks_df["label"] = "ADHD"
control_chunks_df["label"] = "Control"

# we need to "renumber" the chunks for the control data so that they are continuous with the ADHD data
control_chunks_df["chunk_number"] += adhd_chunks_df["chunk_number"].max() + 1

all_data_df = pd.concat([adhd_chunks_df, control_chunks_df], ignore_index=True)

print(all_data_df["chunk_number"].nunique())

4173


In [10]:
chunks_df = (
    all_data_df[["chunk_number", "subject_id", "label"]]
    .drop_duplicates()
    .reset_index(drop=True)
)
chunks_df = pd.get_dummies(chunks_df, columns=["label"])
print(chunks_df.shape)

(4173, 4)


In [11]:
# Let's reshape the data into a 3D array (chunks x channels x samples)
import einops

raw_data = all_data_df.drop(columns=["subject_id", "label"])
# raw_data.info()

# 1. Extract the data (excluding the 'chunk_number' column)
raw_data = raw_data.iloc[:, :-1].values  # shape will be (#tot_samples, 19)

# 2. Reshape the data into a 3D array
num_chunks = chunks_df.shape[0]
eeg_data = einops.rearrange(
    raw_data,
    "(chunks points) electrodes -> chunks electrodes points",
    chunks=num_chunks,
    points=chunk_size,
)

print(eeg_data.shape)

(4173, 19, 512)


## Model

### Prepare the data for model training

In [12]:
X = eeg_data
# expand the dimensions to make it compatible with the Conv1D layer in Keras
# the new shape will be (#tot_samples, 1, 19, 512)
X = np.expand_dims(X, axis=1)

y = chunks_df[["label_ADHD", "label_Control"]].values

# Set the random seed for reproducibility
np.random.seed(42)

# 1. Generate a random permutation of the indices
indices = np.random.permutation(len(X))

# 2. Use the first 80% of the indices for the training set
# train_size = int(0.8 * len(X))
train_size = 3695

# 3. Split the indices into train and test sets
train_indices = indices[:train_size]
test_indices = indices[train_size:]

X_train = X[train_indices]
X_test = X[test_indices]

y_train = y[train_indices]
y_test = y[test_indices]

chunks_train = chunks_df.iloc[train_indices].values
chunks_test = chunks_df.iloc[test_indices].values

print(X_train.shape, y_train.shape, chunks_train.shape)
print(X_test.shape, y_test.shape, chunks_test.shape)

(3695, 1, 19, 512) (3695, 2) (3695, 4)
(478, 1, 19, 512) (478, 2) (478, 4)


In [13]:
# Transform all labels to proper one-hot encoding (integers instead of booleans) - not needed
# y_train = y_train.astype(int)
# y_test = y_test.astype(int)

Group_train = chunks_train[:, [0, 1]]
Group_test = chunks_test[:, [0, 1]]

In [14]:
label_distr_counts_train = np.sum(y_train, axis=0)
label_distr_counts_test = np.sum(y_test, axis=0)

print(f"Training set: {label_distr_counts_train[0]} ADHD /  {label_distr_counts_train[1]} Control")
print(f"Test set: {label_distr_counts_test[0]} ADHD /  {label_distr_counts_test[1]} Control")

Training set: 2072 ADHD /  1623 Control
Test set: 258 ADHD /  220 Control


### Build the model

In [15]:
from tensorflow.keras import backend as K

K.set_image_data_format("channels_first")

In [17]:
# from pyriemann.utils.viz import plot_confusion_matrix
from sklearn.metrics import fbeta_score

def subject_classification1(y_true, y_pred, group, calculate_type="max_vote"):
    """
    y_pred should be the output of 'model.predict' using 2 nodes in the dense layer and softmax activation.
    y_true should be in categorical mode.
    """

    # Categorical to normal labeling
    # NB: 0 will indicate ADHD and 1 will indicate Control (TO BE CHECKED)
    y_true = y_true.argmax(axis=-1)    
    probability = np.array(y_pred)
    # Turn predictions to one hot
    max_indices = np.argmax(y_pred, axis=1)
    prediction_one_hot = np.zeros((y_pred.shape[0], y_pred.shape[1]))
    prediction_one_hot[np.arange(y_pred.shape[0]), max_indices] = 1
    
    max_vote = []
    subject_target = []

    # Combine prediction probabilities and subject labels
    probability = np.concatenate((probability, group), axis=1)
    # Convert combined array to DataFrame
    probability_df = pd.DataFrame(probability, columns=['ADHD', 'Control', 'Subject'])
    # Make sure the prediction columns are float
    probability_df[['ADHD', 'Control']] = probability_df[['ADHD', 'Control']].astype(float)
    # Compute mean of predictions per unique subject
    mean_predictions_per_subject = probability_df.groupby('Subject').mean()
    # Make sure the the groups are sorted by the grouping key (the subject id) after grouping
    mean_predictions_per_subject = mean_predictions_per_subject.sort_index()
    # Revert the DataFrame to a numpy array
    mean_predictions_per_subject = mean_predictions_per_subject.to_numpy()
    mean_ = mean_predictions_per_subject.argmax(axis=-1)

    # Combine prediction classes and subject labels
    prediction = np.concatenate((prediction_one_hot, group), axis=1)
    # Convert combined array to DataFrame
    prediction_df = pd.DataFrame(prediction, columns=['ADHD', 'Control', 'Subject'])
    # Make sure the prediction columns are int
    prediction_df[['ADHD', 'Control']] = prediction_df[['ADHD', 'Control']].astype(int)
    # Compute total number a class was predicted per unique subject
    sum_predictions_per_subject = prediction_df.groupby('Subject').sum()
    # # Make sure the the groups are sorted by the grouping key (the subject id) after grouping
    sum_predictions_per_subject = sum_predictions_per_subject.sort_index()
    # Revert the DataFrame to a numpy array
    sum_predictions_per_subject = sum_predictions_per_subject.to_numpy()
    # sum_predictions_per_subject
    max_vote = np.argmax(sum_predictions_per_subject, axis=1)
    subject_target = np.array(y_true)

    # # j = 0
    # # unique, counts = np.unique(group, return_counts=True)
    # # mean_ = np.zeros([len(unique), 2], dtype="float32")
    # # for i in range(len(unique)):
    # #     for k in range(2):
    # #         mean_[i][k] = np.mean(probability[j : j + counts[i] - 1, k])
    # #     c = np.bincount(prediction[j : j + counts[i] - 1])
    # #     max_vote.append(np.argmax(c))
    # #     subject_traget.append(y_true[j])
    # #     j = j + counts[i]
    # # mean_ = mean_.argmax(axis=-1)
    # # max_vote = np.array(max_vote)
    # # subject_traget = np.array(subject_traget)

    f2_max_vote = fbeta_score(subject_target, max_vote, beta=0.5, average="binary")
    f2_mean = fbeta_score(subject_target, mean_, beta=0.5, average="binary")
    acc_max_vote = np.mean(max_vote == subject_target)
    acc_mean = np.mean(mean_ == subject_target)
    if calculate_type == 'max_vote':
        return acc_max_vote, f2_max_vote
    elif calculate_type == 'mean':
        return acc_mean, f2_mean
    else:
        raise ValueError('You have NOT entered an accurate type!')

In [18]:
def augment_time_series_data(data, label, sigma, magnification_factor):
    augmented_data = []
    augmented_label = []

    n, _, electrodes, time_steps = data.shape
    if sigma[0] == 0 and magnification_factor[0] == 0:
        return data, label
    else:
        # loop through each sample
        for i in range(n):
            for s in sigma:
                for m in magnification_factor:
                    # generate noise with the given sigma
                    noise = np.random.normal(0, s, (electrodes, time_steps))

                    # augment the data with the noise and magnification factor
                    augmented_sample = data[i, :, :, :] + noise * m
                    augmented_data.append(augmented_sample)

                    # augment the label with the same label
                    augmented_label.append(label[i])
        # change list to numpy array
        augmented_data, augmented_label = np.array(augmented_data), np.array(
            augmented_label
        )

        return np.concatenate((data, augmented_data)), np.concatenate(
            (label, augmented_label)
        )

In [19]:
import itertools
# from itertools import combinations

def generate_combinations(magnification_factor, sigma):
    result = [{"magnification_factor": [0], "sigma": [0]}]
    for i in range(1, 4):
        for m_comb in itertools.combinations(magnification_factor, i):
            for sigma_comb in itertools.combinations(sigma, i):
                result.append(
                    {"magnification_factor": list(m_comb), "sigma": list(sigma_comb)}
                )
    return result

In [20]:
def sample_classification(y_true, y_pred):
    """
    y_pred should be the output of 'model.predict' using 2 nodes in the dense layer and softmax activation.
    y_true should be in categorical mode.
    """
    preds = y_pred.argmax(axis=-1)
    acc = np.mean(preds == y_true.argmax(axis=-1))
    f2 = fbeta_score(y_true.argmax(axis=-1), preds, beta=0.5, average="binary")
    return acc, f2

In [21]:
def ClassWeightSoftmax(y):
    class_weights = compute_class_weight(
        class_weight="balanced", classes=np.unique(Y_train), y=Y_train
    )
    class_weights = dict(zip(np.unique(Y_train), class_weights))
    class_weights[0] = int(10 * round(class_weights[0], 1))
    class_weights[1] = int(10 * round(class_weights[1], 1))
    class_weight = [{0: 1, 1: class_weights[0]}, {0: 1, 1: class_weights[1]}]
    print("The Class Weight is:", class_weight)
    return class_weight

In [22]:
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras import utils as np_utils
from sklearn.model_selection import StratifiedGroupKFold, GridSearchCV
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout, multiply, LSTM
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler

# from tensorflow.keras.optimizers import Adam, Adamax
# from tensorflow.keras.optimizers.experimental import AdamW
from tensorflow.keras.optimizers import Adam, Adamax, AdamW
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import GlobalAveragePooling2D
from keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import fbeta_score
import math
import gc
from numpy import mean
from numpy import std
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from sklearn.model_selection import cross_val_score
from skopt.utils import use_named_args
from skopt import gp_minimize
import tensorflow as tf
import numpy as np


def TactileNet(
    nb_classes=2,
    Chans=19,
    Samples=512,
    kernLength=16,
    F2=64,
    F1=64,
    D=4,
    dropoutRate=0.5,
    dropoutType="Dropout",
    norm_rate=0.25,
    Dense_nodes=16,
    optimizer_type="Adam",
    lr=0.001,
    **kwargs
):

    if dropoutType == "SpatialDropout2D":
        dropoutType = SpatialDropout2D
    elif dropoutType == "Dropout":
        dropoutType = Dropout
    else:
        raise ValueError(
            "dropoutType must be one of SpatialDropout2D "
            "or Dropout, passed as a string."
        )
    # EEGNet alike part
    input1 = Input(shape=(1, Chans, Samples))
    block1 = Conv2D(
        F1,
        (1, kernLength),
        padding="same",
        input_shape=(1, Chans, Samples),
        use_bias=False,
    )(input1)
    block1 = BatchNormalization(axis=1, trainable=True)(block1)
    block1 = DepthwiseConv2D(
        (Chans, 1),
        use_bias=False,
        depth_multiplier=D,
        depthwise_constraint=max_norm(norm_rate),
    )(block1)
    block1 = BatchNormalization(axis=1, trainable=True)(block1)
    block1 = Activation("elu")(block1)
    block1 = AveragePooling2D((1, 2))(block1)
    block1 = dropoutType(dropoutRate)(block1)

    ###############################################
    # first tower
    sub_block1 = Conv2D(64, (1, 1), padding="same", use_bias=False)(block1)
    sub_block1 = SeparableConv2D(128, (1, 128), padding="same", use_bias=False)(
        sub_block1
    )
    sub_block1 = AveragePooling2D((1, 2), padding="same")(sub_block1)
    # second tower
    sub_block2 = Conv2D(16, (1, 1), padding="same", use_bias=False)(block1)
    sub_block2 = SeparableConv2D(32, (1, 256), padding="same", use_bias=False)(
        sub_block2
    )
    sub_block2 = AveragePooling2D((1, 2), padding="same")(sub_block2)
    # third tower
    sub_block3 = Conv2D(64, (1, 1), padding="same", strides=(1, 2), use_bias=False)(
        block1
    )
    # forth tower
    sub_block4 = AveragePooling2D((1, 2), padding="same")(block1)
    sub_block4 = Conv2D(32, (1, 1), padding="same", use_bias=False)(sub_block4)
    # concatenation
    concat = concatenate([sub_block1, sub_block2, sub_block4, sub_block3], axis=1)

    # last tower
    block2 = BatchNormalization(axis=1, trainable=True)(concat)
    block2 = Activation("elu")(block2)
    # SENEt block
    squeeze1 = GlobalAveragePooling2D()(block2)
    excitation1 = Dense(Dense_nodes, activation="relu")(squeeze1)
    excitation1 = Dense(256, activation="sigmoid")(excitation1)
    block2 = Permute(dims=(2, 3, 1))(block2)
    excitation1 = multiply([block2, excitation1])
    excitation1 = Permute(dims=(3, 1, 2))(excitation1)

    block2 = SeparableConv2D(256, (1, 64), padding="same", use_bias=False)(excitation1)
    block2 = BatchNormalization(axis=1, trainable=True)(block2)
    block2 = Activation("elu")(block2)
    # SENEt block
    squeeze2 = GlobalAveragePooling2D()(block2)
    excitation2 = Dense(Dense_nodes, activation="relu")(squeeze2)
    excitation2 = Dense(256, activation="sigmoid")(excitation2)
    block2 = Permute(dims=(2, 3, 1))(block2)
    excitation2 = multiply([block2, excitation2])
    excitation2 = Permute(dims=(3, 1, 2))(excitation2)

    block2 = dropoutType(dropoutRate)(excitation2)

    GB = GlobalAveragePooling2D()(block2)
    denselayer = Dense(nb_classes, name="denselayer", kernel_constraint=max_norm(norm_rate))(GB)
    softmax = Activation("softmax", name="softmax")(denselayer)
    if optimizer_type == "Adam":
        optimizer = Adam(learning_rate=lr)
    if optimizer_type == "Adamax":
        optimizer = Adamax(learning_rate=lr)
    if optimizer_type == "AdamW":
        optimizer = AdamW(learning_rate=lr)
    model = Model(inputs=input1, outputs=softmax)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["accuracy"])
    return model

### Training

In [23]:
model = TactileNet()
print(model.summary())

/Users/marco/miniconda3/envs/msc2024_eeg-tf/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-09-01 21:00:35.632295: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2024-09-01 21:00:35.632314: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-09-01 21:00:35.632324: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-09-01 21:00:35.632345: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-01 21:00:35.632356: I tensorflow/core/common_runtime/pluggable_device/pluggable

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1, 19,     │          0 │ -                 │
│ (InputLayer)        │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 64, 19,    │      1,024 │ input_layer[0][0] │
│                     │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 64, 19,    │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ depthwise_conv2d    │ (None, 256, 1,    │      4,864 │ batch_normalizat… │
│ (DepthwiseConv2D)   │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 256, 1,    │      1,024 │ depthwise_conv2d… │
│ (BatchNormalizatio… │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation          │ (None, 256, 1,    │          0 │ batch_normalizat… │
│ (Activation)        │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling2d   │ (None, 256, 1,    │          0 │ activation[0][0]  │
│ (AveragePooling2D)  │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256, 1,    │          0 │ average_pooling2… │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 64, 1,     │     16,384 │ dropout[0][0]     │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 16, 1,     │      4,096 │ dropout[0][0]     │
│                     │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d    │ (None, 128, 1,    │     16,384 │ conv2d_1[0][0]    │
│ (SeparableConv2D)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ separable_conv2d_1  │ (None, 32, 1,     │      4,608 │ conv2d_2[0][0]    │
│ (SeparableConv2D)   │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling2d_3 │ (None, 256, 1,    │          0 │ dropout[0][0]     │
│ (AveragePooling2D)  │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling2d_1 │ (None, 128, 1,    │          0 │ separable_conv2d… │
│ (AveragePooling2D)  │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ average_pooling2d_2 │ (None, 32, 1,     │          0 │ separable_conv2d… │
│ (AveragePooling2D)  │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 1,     │      8,192 │ average_pooling2… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 1,     │     16,384 │ dropout[0][0]   

 Total params: 174,626 (682.13 KB)

 Trainable params: 172,962 (675.63 KB)

 Non-trainable params: 1,664 (6.50 KB)

None


In [24]:
# model.fit(X_train, y_train, epochs=5, batch_size=32, verbose=1)

In [25]:
# define the space of hyperparameters to search
search_space = list()
search_space.append(Categorical([8, 16, 32, 64], name="F1"))
search_space.append(Categorical([16, 32, 64, 128, 256], name="F2"))
search_space.append(Categorical([2, 4, 8], name="D"))
search_space.append(Categorical([32, 64, 128, 256, 512], name="kernLength"))
search_space.append(Categorical([8, 16, 32], name="Dense_nodes"))
search_space.append(Categorical([0.5, 1.0, 5.0, 10.0], name="norm_rate"))
search_space.append(Categorical([0.0001, 0.0005, 0.001, 0.005, 0.01], name="lr"))
search_space.append(Categorical(["Adam", "Adamax", "AdamW"], name="optimizer"))
search_space.append(Categorical([0.3, 0.4, 0.5], name="dropoutRate"))
search_space.append(Categorical([32, 64, 128], name="batch_size"))


# define the function used to evaluate a given configuration
@use_named_args(search_space)
def evaluate_model(**params):
    sample_acc = []
    sample_f2 = []
    subject_acc = []
    subject_f2 = []
    sample_loss = []
    ### Defining SKGF ###
    #######################
    # Callbacks #
    # modelpath = "./Bayesian Saved Model/model_{epoch:02d}_{val_loss:.2f}.keras"
    model_path = "./Bayesian Saved Model/model.keras"
    checkpoint = ModelCheckpoint(
        model_path, monitor="val_accuracy", verbose=1, save_best_only=True, mode="max"
    )
    es = EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=15)
    keras_reduce_lr = ReduceLROnPlateau(
        monitor="val_loss", factor=0.5, patience=10, min_lr=0.00005, verbose=2
    )
    callbacks = [checkpoint, es, keras_reduce_lr]
    model = TactileNet(**params)
    # Fitting ...
    fit = model.fit(
        X_train,
        y_train,
        verbose=2,
        shuffle=True,
        # epochs=100,
        epochs=2, # changed to keep training short
        batch_size=params["batch_size"],
        validation_data=(X_test, y_test),
        callbacks=callbacks,
    )

    del model, fit

    ### Loading The Best Saved Model ###
    saved_model = load_model(model_path)
    
    # Predict The model
    test_predict = saved_model.predict(X_test)
    test_loss, test_acc = saved_model.evaluate(X_test, y_test)
    ######################################################################################################
    print("test_loss", test_loss)
    print("test_acc", test_acc)
    ######################################################################################################

    # Select the true labels for the (unique) subjects
    _, indices = np.unique(Group_test[:, 1], return_index=True)
    subject_true_pred = y_test[indices]

    Group_test_labels = Group_test[:, 1].reshape(-1, 1)

    sub_acc, sub_f2 = subject_classification1(
        subject_true_pred, test_predict, Group_test_labels, calculate_type="max_vote"
    )
    sam_acc, sam_f2 = sample_classification(y_test, test_predict)

    # convert from a maximizing score to a minimizing score
    return 1.0 - sam_acc

In [26]:
# DEBUG
DB_model_path = "./Bayesian Saved Model/model.keras"
DB_saved_model = load_model(DB_model_path)

# Select the true labels for the (unique) subjects
_, indices = np.unique(Group_test[:, 1], return_index=True)
DB_y_true = y_test[indices]

DB_y_pred = DB_saved_model.predict(X_test)
DB_group = Group_test[:, 1].reshape(-1, 1)

DB_subj_res = subject_classification1(DB_y_true, DB_y_pred, DB_group, calculate_type="max_vote")
print(DB_subj_res)

DB_sam_res = sample_classification(y_test, DB_y_pred)
print(DB_sam_res)

2024-09-01 21:00:36.515619: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 32ms/step
(0.896551724137931, 0.9504132231404959)
(0.895397489539749, 0.9334763948497854)


In [27]:
# n_iteration = 100
n_iteration = 10
# perform optimization
result = gp_minimize(evaluate_model, search_space, n_calls=n_iteration)
# summarizing finding:
print("Best Accuracy: %.3f" % (1.0 - result.fun))
print("Best Parameters: %s" % (result.x))

/Users/marco/miniconda3/envs/msc2024_eeg-tf/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/2

Epoch 1: val_accuracy improved from -inf to 0.46234, saving model to ./Bayesian Saved Model/model.keras
29/29 - 61s - 2s/step - accuracy: 0.7396 - loss: 0.5227 - val_accuracy: 0.4623 - val_loss: 0.6746 - learning_rate: 5.0000e-04
Epoch 2/2

Epoch 2: val_accuracy improved from 0.46234 to 0.47280, saving model to ./Bayesian Saved Model/model.keras
29/29 - 57s - 2s/step - accuracy: 0.8723 - loss: 0.3046 - val_accuracy: 0.4728 - val_loss: 0.6732 - learning_rate: 5.0000e-04
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 34ms/step - accuracy: 0.4634 - loss: 0.6772
test_loss 0.6731755137443542
test_acc 0.47280335426330566
Epoch 1/2


/Users/marco/miniconda3/envs/msc2024_eeg-tf/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)



Epoch 1: val_accuracy improved from -inf to 0.83891, saving model to ./Bayesian Saved Model/model.keras
29/29 - 80s - 3s/step - accuracy: 0.7716 - loss: 0.4667 - val_accuracy: 0.8389 - val_loss: 0.6683 - learning_rate: 5.0000e-04
Epoch 2/2

Epoch 2: val_accuracy improved from 0.83891 to 0.88703, saving model to ./Bayesian Saved Model/model.keras
29/29 - 77s - 3s/step - accuracy: 0.9234 - loss: 0.1953 - val_accuracy: 0.8870 - val_loss: 0.6416 - learning_rate: 5.0000e-04
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 49ms/step - accuracy: 0.8906 - loss: 0.6435
test_loss 0.6416162848472595
test_acc 0.8870292901992798


/Users/marco/miniconda3/envs/msc2024_eeg-tf/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

Epoch 1/2

Epoch 1: val_accuracy improved from -inf to 0.48954, saving model to ./Bayesian Saved Model/model.keras
29/29 - 71s - 2s/step - accuracy: 0.8300 - loss: 0.3825 - val_accuracy: 0.4895 - val_loss: 2.9816 - learning_rate: 0.0100
Epoch 2/2

Epoch 2: val_accuracy improved from 0.48954 to 0.72176, saving model to ./Bayesian Saved Model/model.keras
29/29 - 65s - 2s/step - accuracy: 0.9405 - loss: 0.1520 - val_accuracy: 0.7218 - val_loss: 0.4922 - learning_rate: 0.0100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.7366 - loss: 0.4719
test_loss 0.49224498867988586
test_acc 0.7217572927474976


/Users/marco/miniconda3/envs/msc2024_eeg-tf/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/2

Epoch 1: val_accuracy improved from -inf to 0.89121, saving model to ./Bayesian Saved Model/model.keras
58/58 - 53s - 916ms/step - accuracy: 0.8888 - loss: 0.2640 - val_accuracy: 0.8912 - val_loss: 0.3172 - learning_rate: 0.0100
Epoch 2/2

Epoch 2: val_accuracy did not improve from 0.89121
58/58 - 45s - 768ms/step - accuracy: 0.9645 - loss: 0.0906 - val_accuracy: 0.8640 - val_loss: 1.5440 - learning_rate: 0.0100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 67ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 2s 64ms/step - accuracy: 0.9020 - loss: 0.2771
test_loss 0.31717172265052795
test_acc 0.8912134170532227


/Users/marco/miniconda3/envs/msc2024_eeg-tf/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/2

Epoch 1: val_accuracy improved from -inf to 0.96234, saving model to ./Bayesian Saved Model/model.keras
116/116 - 36s - 310ms/step - accuracy: 0.9012 - loss: 0.2282 - val_accuracy: 0.9623 - val_loss: 0.1135 - learning_rate: 0.0100
Epoch 2/2

Epoch 2: val_accuracy did not improve from 0.96234
116/116 - 27s - 237ms/step - accuracy: 0.9591 - loss: 0.1141 - val_accuracy: 0.9289 - val_loss: 0.3176 - learning_rate: 0.0100
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 54ms/step
15/15 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - accuracy: 0.9629 - loss: 0.1211
test_loss 0.11351849883794785
test_acc 0.9623430967330933
Best Accuracy: 0.962
Best Parameters: [16, 16, 8, 32, 8, 1.0, 0.01, 'AdamW', 0.5, 32]


### Save results

In [28]:
# # DEBUG

# import pickle

# # Save the object to a file
# with open('optimized_result.pkl', 'wb') as file:
#     pickle.dump(result, file)

In [29]:
# import dill

# # Save the entire workspace (all variables)
# with open('workspace.pkl', 'wb') as f:
#     dill.dump_session(f)